In [93]:
#imports
try:
    import re
    import json
    import codecs
    from pprint import pprint
    from bs4 import BeautifulSoup
    from urllib.request import quote
    from urllib.request import urlopen
except ImportError:
    print("Error while loading libraries...")
except Exception:
    print('Uncaught exception')
else:
    print("Libraries loaded.")

Libraries loaded.


In [94]:
#constants
mainsite = "http://rozklad.nau.edu.ua/timetable/group/"
NNIIDS121 = "http://rozklad.nau.edu.ua/timetable/group/%D0%9D%D0%9D%D0%86%D0%86%D0%94%D0%A1%20121"
lessons = []

In [95]:
#Returns html for all groups
def scrapeGroups():
    allScrapedGroups = []
    for s in groupNamesEncoded:
#         print(mainsite+s)
        nau = urlopen(mainsite+s)
        allScrapedGroups.append(BeautifulSoup(nau, 'html.parser'))
    return allScrapedGroups

In [96]:
# 1. -> 08:00 - 09:35
# 2. -> 09:50 - 11:25
# 3. -> 11:40 - 13:15
# 4. -> 13:30 - 15:05
# 5. -> 15:20 - 16:55
# 6. -> 17:10 - 18:45
# 7. -> 19:00 - 20:35
# 8. -> 20:50 - 22:25

def paraToTime(num):
    schedule = [
        "08:00 - 09:35",
        "09:50 - 11:25",
        "11:40 - 13:15",
        "13:30 - 15:05",
        "15:20 - 16:55",
        "17:10 - 18:45",
        "19:00 - 20:35",
        "20:50 - 22:25"]
    return schedule[num]

In [97]:
def extractTags(soupMain):
    return list(map(lambda x: x.text, soupMain.findChildren('select', attrs={'name': 'institute'})))[0].split('\n')[2:-1]

In [98]:
def encodeName(lessonNames):
    return list(map(lambda x: quote(x.encode('UTF-8')), lessonNames))

In [99]:
def getWeeks():
    week1 = soup.findAll('td', attrs={'colspan': '3'})[1:37]
    week2 = soup.findAll('td', attrs={'colspan': '3'})[38:-1]

In [158]:
#Complete JSON of a lesson
def makeObj(i,tag):
    if not tag.contents: 
        return {}
    a=0
    if i < 35:
       a = 1
    else: 
        a = 0
    index = tag.small.contents[0].find(' ')
    housing = 'undefined'
    room_num = 'undefined'
    room_floor = 'undefined'
    if not re.findall('\d+',tag.contents[0]):
        housing = ('C/K')
        room_num = ""
        room_floor = ""
    else: 
        housing = re.findall('\d+',tag.contents[0])[0]
        room_num = re.findall('\d+',tag.contents[0])[1]
        room_floor = re.findall('\d+',tag.contents[0])[1][0]
    return {
        'name': tag.b.contents[0], 
        'short_name': tag.b.contents[0], 
        'lesson_time':paraToTime(i%6),
        'day': int(i/6)+1,
        'format': lessonFormat(tag),
        'teachers': [{
                  "degree":"",
                  "last_name": tag.small.contents[0][0:index],
                  "first_name": tag.small.contents[0][index+1:index+2],
                  "middle_name": tag.small.contents[0][index+3:index+4]
               } ], 
        'weeks':a,
        'housing':housing,
        'room_num':room_num,
        'room_floor':room_floor}


In [110]:
#returns lecture or practice
def lessonFormat(tag):
    if 'class' in tag.attrs:
        return 'lecture'
    else:
        return 'practice'

In [111]:
nauMain = urlopen(mainsite)
soupMain = BeautifulSoup(nauMain, 'html.parser')

In [112]:
try:
    nau = urlopen(NNIIDS121)
except ConnectionError:
    print('ConnectionError')
except NameError:
    print('NameError')
except Exception:
    print('Uncaught exception')
else:
    print('HTTPResponse object loaded to variable')

HTTPResponse object loaded to variable


In [113]:
facultyNames = extractTags(soupMain)
print(facultyNames)

['ІІКАО', 'ДП  ННІН', 'ЗДН ННІН', 'ННІІДС', 'ННІАН', 'ННІАП', 'ННІЕБ', 'ННІЕМ', 'ННІКІТ', 'ННІМВ', 'ННАКІ', 'ННГМІ', 'ННЮІ', 'ПДН ННІН', 'ФТТ', 'ЦМО', 'ЦНТ', 'ЦНТ н-1']


In [114]:
groupNamesEncoded = encodeName(facultyNames)
print(groupNamesEncoded)

['%D0%86%D0%86%D0%9A%D0%90%D0%9E', '%D0%94%D0%9F%20%20%D0%9D%D0%9D%D0%86%D0%9D', '%D0%97%D0%94%D0%9D%20%D0%9D%D0%9D%D0%86%D0%9D', '%D0%9D%D0%9D%D0%86%D0%86%D0%94%D0%A1', '%D0%9D%D0%9D%D0%86%D0%90%D0%9D', '%D0%9D%D0%9D%D0%86%D0%90%D0%9F', '%D0%9D%D0%9D%D0%86%D0%95%D0%91', '%D0%9D%D0%9D%D0%86%D0%95%D0%9C', '%D0%9D%D0%9D%D0%86%D0%9A%D0%86%D0%A2', '%D0%9D%D0%9D%D0%86%D0%9C%D0%92', '%D0%9D%D0%9D%D0%90%D0%9A%D0%86', '%D0%9D%D0%9D%D0%93%D0%9C%D0%86', '%D0%9D%D0%9D%D0%AE%D0%86', '%D0%9F%D0%94%D0%9D%20%D0%9D%D0%9D%D0%86%D0%9D', '%D0%A4%D0%A2%D0%A2', '%D0%A6%D0%9C%D0%9E', '%D0%A6%D0%9D%D0%A2', '%D0%A6%D0%9D%D0%A2%20%D0%BD-1']


In [104]:
a = scrapeGroups()

In [115]:
try:
    soup = BeautifulSoup(nau, 'html.parser')
except ImportError:
    print('ImportError')

In [159]:
week1 = soup.findAll('td', attrs={'colspan': '3'})[1:37]
week2 = soup.findAll('td', attrs={'colspan': '3'})[38:-1]
x = week1+week2

In [160]:
y =  [makeObj(z,x[z]) for z in range(0,71)]

In [161]:
pprint(y)

[{'day': 1,
  'format': 'practice',
  'housing': '11',
  'lesson_time': '08:00 - 09:35',
  'name': 'Вища математика',
  'room_floor': '2',
  'room_num': '214',
  'short_name': 'Вища математика',
  'teachers': [{'degree': '',
                'first_name': 'С',
                'last_name': 'Глоба',
                'middle_name': 'О'}],
  'weeks': 1},
 {'day': 1,
  'format': 'practice',
  'housing': '11',
  'lesson_time': '09:50 - 11:25',
  'name': 'Історія та культура України',
  'room_floor': '2',
  'room_num': '214',
  'short_name': 'Історія та культура України',
  'teachers': [{'degree': '',
                'first_name': 'Н',
                'last_name': 'Сенчило',
                'middle_name': 'О'}],
  'weeks': 1},
 {},
 {},
 {},
 {},
 {},
 {},
 {'day': 2,
  'format': 'lecture',
  'housing': '11',
  'lesson_time': '11:40 - 13:15',
  'name': 'Вища математика',
  'room_floor': '3',
  'room_num': '311',
  'short_name': 'Вища математика',
  'teachers': [{'degree': '',
                'f

In [157]:
k = [k for k in y if len(k)>0]
k = {"ННІІДС":{"lessons": k, "subgroups":"0", "teachers": []}}
with codecs.open("NAU.json", "w", encoding='utf-8') as out:
    json.dump(k, out, ensure_ascii=False, indent=4)

In [122]:
# pprint(k)